# Surprise Minimization – Experiment 01This notebook illustrates the continuous PDE model:$$\partial_t S = -1 + \alpha\Delta S,$$with Neumann boundary conditions implemented via symmetric padding.The simulation shows:* transient curvature (exploration),* decay of predictive gradients (collapse),* dark-room asymptotics (steady state),* exploratory phase as **simulated danger**,* collapse as **inoculation against surprise**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# Domain and simulation params
nx = 120
alpha = 0.1           # diffusion strength
steps = 2000
plot_interval = 400   # refresh figures periodically
dx = 1.0
dt = 0.1
# CFL condition (stable explicit scheme)
if dt > dx**2/(2*alpha*2):
    print('Warning: dt may violate CFL for 2D explicit diffusion')

In [ ]:
# Create a 2D Laplacian operator using convolution
from scipy.signal import convolve2d
kernel = np.array([[0, 1, 0],
                   [1,-4, 1],
                   [0, 1, 0]]) / dx**2
def laplace(S):
    # symmetric padding implements Neumann BC visually
    return convolve2d(S, kernel, mode='same', boundary='symm')

In [ ]:
# Initial condition (random or spatial structure) 
S = np.random.randn(nx, nx) * 0.5
# Optionally experiment: sinusoidal initial condition for metastability
# x = np.linspace(0,1,nx)
# X,Y = np.meshgrid(x,x)
# S = np.sin(2*np.pi*X) * np.sin(2*np.pi*Y)
def step(S):
    return S + dt*( -1 + alpha*laplace(S) )

### Helper: curvature measurementWe measure epistemic curvature as the L2 norm of the Laplacian or gradient, illustrating collapse.

In [ ]:
curvature_history = []for t in range(steps):
    S = step(S)
    curv = np.sqrt(np.mean(laplace(S)**2))
    curvature_history.append(curv)

    if t % plot_interval == 0:
        plt.figure(figsize=(5,4))
        plt.imshow(S, cmap='viridis')
        plt.title(f'Surprise field S at step {t}')
        plt.colorbar()
        plt.show()

### Curvature decay (epistemic collapse)This demonstrates the **dark-room limit**: after initial transients (exploration), the curvature decays toward zero, representing predictive flattening and policy collapse.

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(curvature_history)
plt.xlabel('Time step')
plt.ylabel('Curvature ||ΔS|| (epistemic)')
plt.title('Epistemic curvature decay (Simulated exploration → Inoculation)')
plt.grid(True)
plt.show()

## Interpretation (in plain language)* At first the system “plays” with epistemic curvature — controlled risk.
* This is **simulated danger**: temporarily increasing uncertainty.
* Then curvature collapses.
* Learning has inoculated the agent against further surprise.
* Without external forcing, the system enters a **dark-room equilibrium**: minimal curvature, minimal surprise, zero policy.